In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


Using TensorFlow backend.


In [ ]:
#https://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras

In [20]:
data = pd.read_csv('./data.csv')
data.columns = ['Class', 'Data']

In [21]:
print(data[ data['Class'] == 1].size)
print(data[ data['Class'] == 0].size)

#for idx,row in data.iterrows():
#    row[0] = row[0].replace('rt',' ')
    
max_fatures = 20000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Data'].values)
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
X = tokenizer.texts_to_sequences(data['Data'].values)
X = pad_sequences(X, maxlen = 250)

7756
9398


In [22]:
print(X.shape)

(8577, 250)


In [23]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 128)          2560000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 250, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 2,815,194
Trainable params: 2,815,194
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#print(data)

In [24]:
Y = pd.get_dummies(data['Class']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5746, 250) (5746, 2)
(2831, 250) (2831, 2)


In [25]:
batch_size = 256
model.fit(X_train, Y_train, epochs = 8, batch_size=batch_size, verbose = 2)

Epoch 1/8
 - 131s - loss: 0.6716 - acc: 0.5687
Epoch 2/8
 - 116s - loss: 0.5100 - acc: 0.7663
Epoch 3/8
 - 112s - loss: 0.3150 - acc: 0.8714
Epoch 4/8
 - 114s - loss: 0.2018 - acc: 0.9264
Epoch 5/8
 - 119s - loss: 0.1331 - acc: 0.9509
Epoch 6/8
 - 117s - loss: 0.0957 - acc: 0.9687
Epoch 7/8
 - 121s - loss: 0.0617 - acc: 0.9800
Epoch 8/8
 - 116s - loss: 0.0463 - acc: 0.9859


In [26]:
validation_size = 1600

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.5f" % (score))
print("acc: %.5f" % (acc))

score: 0.66254
acc: 0.81885


In [27]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 81.4404432132964 %
neg_acc 82.46013667425967 %


In [28]:
twt = ['আমি খারাপ আছি']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=250, dtype='int32', value=0)
#print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

negative


In [29]:
model.save('./senti.h5')

In [19]:
from keras import backend as K
K.clear_session()